In [1]:
import numpy as np
from sklearn.utils import check_random_state
from scipy.signal import convolve
from time import time

In [2]:
def get_random_size(random_state=None, N=1000):
    rng = check_random_state(random_state)
    idx1 = rng.randint(N)
    idx2 = rng.randint(N)
    Ns = np.logspace(np.log10(3), np.log10(200), num=N).astype(int)
    s1 = Ns[idx1]
    s2 = Ns[idx2]
    return (s1, s1), (s2, s2)

def _fftconv_faster(S1, S2, mode="full", random_state=None):
    rng = check_random_state(random_state)
    x = rng.randn(*S1)
    h = rng.randn(*S2)
    
    start = time()
    _ = convolve(x, h, mode=mode, method="fft")
    fft_time = time() - start
    
    start = time()
    _ = convolve(x, h, mode=mode, method="direct")
    direct_time = time() - start
    return (
        "fft" if fft_time < direct_time else "direct",
        {"fft_time": fft_time, "direct_time": direct_time},
        (x, h),
    )

In [3]:
import math
def _prod(iterable):
    """
    Product of a list of numbers.
    Faster than np.prod for short lists like array shapes.
    """
    product = 1
    for x in iterable:
        product *= x
    return product
def _predict_fftconv_faster(x, h, constant, mode="full"):
    if mode == 'full':
        out_shape = [n + k - 1 for n, k in zip(x.shape, h.shape)]
    elif mode == 'same':
        out_shape = x.shape
    elif mode == 'valid':
        out_shape = [n - k + 1 for n, k in zip(x.shape, h.shape)]
    else:
        raise ValueError('mode is invalid')

    # see whether the Fourier transform convolution method or the direct
    # convolution method is faster (discussed in scikit-image PR #1792)
    direct_time = (x.size * h.size * _prod(out_shape))
    fft_time = sum(n * math.log(n) for n in (x.shape + h.shape +
                                             tuple(out_shape)))
    return "fft" if constant * fft_time < direct_time else "direct"
    

In [4]:
import pandas as pd
data = []
constants = {"full": 81261.988432}
mode = "full"
constant = constants[mode]
for seed in range(1000):
    S1, S2 = get_random_size(random_state=seed)
    fastest, times, (x, h) = _fftconv_faster(S1, S2, random_state=seed, mode=mode)
    predicted_fastest = _predict_fftconv_faster(x, h, constant)
    datum = {
        "shape1": S1,
        "shape2": S2,
        "actual_fastest": fastest,
        "predicted_fastest": predicted_fastest,
        "mode": mode,
        "constant": constant,
        **times,
    }
    data.append(datum)
    if seed % 10 == 0:
        today = "2019-07-27"
        df = pd.DataFrame(data)
        df.to_csv(f"out/{today}-test.csv", index=False)
        print(seed)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [5]:
df.head()

,actual_fastest,constant,mode,predicted_fastest,shape1,shape2
0,fft,81261.988432,full,fft,"(53, 53)","(31, 31)"
1,direct,81261.988432,full,direct,"(3, 3)","(8, 8)"
2,direct,81261.988432,full,direct,"(6, 6)","(27, 27)"
3,fft,81261.988432,full,fft,"(118, 118)","(48, 48)"
4,direct,81261.988432,full,direct,"(5, 5)","(6, 6)"


In [11]:
(df.actual_fastest == df.predicted_fastest).sum() / len(df)

0.5